<a href="https://colab.research.google.com/github/marlongarrido7/ConversorAD/blob/main/Uva1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
from google.colab import files

# Solicita o upload do arquivo ZIP (compactado com a pasta contendo "imagens" e "labels")
uploaded = files.upload()

# Pegue o nome do arquivo enviado (assumindo que há somente um arquivo ZIP)
zip_filename = list(uploaded.keys())[0]
print("Arquivo recebido:", zip_filename)

# Cria um diretório para extrair o conteúdo do ZIP
extract_path = '/content/dataset_original'
os.makedirs(extract_path, exist_ok=True)

# Extrai o arquivo ZIP
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extração concluída!")


CÉLULA 2: Divisão Automática dos Dados (Treino e Validação)

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define o caminho base conforme a estrutura retornada pelo comando !find
base_extracted = os.path.join('/content/dataset_original', "dataset_original")

# Caminhos das pastas de imagens e labels conforme extraídos
ImagemGeral_path = os.path.join(base_extracted, "Imagem Geral")
labelsgeral_path = os.path.join(base_extracted, "labels geral")

# Verifica se as pastas existem
if not os.path.isdir(ImagemGeral_path) or not os.path.isdir(labelsgeral_path):
    raise Exception("Não foi possível encontrar as pastas 'Imagem Geral' ou 'labels geral'. Verifique a estrutura do ZIP e a extração.")

print("Estrutura de pastas verificada:")
print("Imagem Geral:", ImagemGeral_path)
print("labels geral:", labelsgeral_path)

# Função para encontrar recursivamente todos os arquivos de imagem em uma pasta
def find_all_images(folder):
    image_files = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_files.append(os.path.join(root, file))
    return image_files

# Lista todos os arquivos de imagem na pasta "Imagem Geral"
all_image_paths = find_all_images(ImagemGeral_path)
print("Total de imagens encontradas:", len(all_image_paths))

if len(all_image_paths) == 0:
    raise Exception("Nenhuma imagem foi encontrada na pasta 'Imagem Geral'.")

# Divide os caminhos de imagem em treino (80%) e validação (20%)
train_images, val_images = train_test_split(all_image_paths, test_size=0.2, random_state=42)

# Define os diretórios de destino para a nova estrutura
base_dest = '/content/dataset/yolo'
train_images_dest = os.path.join(base_dest, 'train', 'images')
train_labels_dest = os.path.join(base_dest, 'train', 'labels')
val_images_dest   = os.path.join(base_dest, 'val', 'images')
val_labels_dest   = os.path.join(base_dest, 'val', 'labels')

# Cria os diretórios de destino (caso não existam)
for folder in [train_images_dest, train_labels_dest, val_images_dest, val_labels_dest]:
    os.makedirs(folder, exist_ok=True)

# Função auxiliar para copiar um arquivo para o destino
def copy_file(src, dst_folder):
    if os.path.exists(src):
        shutil.copy2(src, os.path.join(dst_folder, os.path.basename(src)))
    else:
        print(f"Aviso: {src} não encontrado.")

# Função para encontrar recursivamente um arquivo de label dado o nome base esperado
def find_label_file(label_folder, label_filename):
    for root, dirs, files in os.walk(label_folder):
        if label_filename in files:
            return os.path.join(root, label_filename)
    return None

# Copia as imagens e seus labels para o conjunto de treino
for img_path in train_images:
    # Copia a imagem
    copy_file(img_path, train_images_dest)

    # Constrói o nome base da imagem e o nome esperado para o label (.txt)
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    label_filename = base_name + ".txt"

    # Procura pelo arquivo de label na pasta "labels geral"
    label_path = find_label_file(labelsgeral_path, label_filename)
    if label_path:
        copy_file(label_path, train_labels_dest)
    else:
        print(f"Aviso: Rótulo para {img_path} não encontrado.")

# Copia as imagens e seus labels para o conjunto de validação
for img_path in val_images:
    copy_file(img_path, val_images_dest)

    base_name = os.path.splitext(os.path.basename(img_path))[0]
    label_filename = base_name + ".txt"
    label_path = find_label_file(labelsgeral_path, label_filename)
    if label_path:
        copy_file(label_path, val_labels_dest)
    else:
        print(f"Aviso: Rótulo para {img_path} não encontrado.")

print("Divisão dos dados concluída!")


CÉLULA 3: Criação do Arquivo YAML para o YOLOv8

In [8]:
%%writefile grape_detection.yaml
# Caminho base para o dataset
# Este caminho se refere ao diretório criado na etapa anterior (/content/dataset/yolo)
path: /content/dataset/yolo

train: train   # Pasta de treino
val: val       # Pasta de validação

nc: 1  # Número de classes (neste exemplo, 1: cacho de uva)
names: ['grape_bunch']


Writing grape_detection.yaml


CÉLULA 4: Treinamento do Modelo com YOLOv8

In [ ]:
# Instale e importe o pacote ultralytics, se ainda não tiver feito
!pip install ultralytics
from ultralytics import YOLO

# Carrega um modelo pré-treinado - aqui usamos a versão "nano" para velocidade
model = YOLO('yolov8n.pt')

# Inicia o treinamento usando o arquivo YAML criado.
# Ajuste o número de épocas (exemplo: 50) e outros hiperparâmetros conforme necessário
results = model.train(data='grape_detection.yaml', epochs=50, imgsz=640)

# Durante o treinamento, serão exibidas as métricas e as imagens com as detecções.


metricas do  modelo

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd

# Exibe os resultados do treinamento:
print("Resultados do Treinamento:")
print(results)  # Imprime o objeto completo de resultados

# Tente extrair as métricas: primeiramente, verifica se há atributo 'results_dict' ou 'metrics'
if hasattr(results, 'results_dict'):
    metrics = results.results_dict
elif hasattr(results, 'metrics'):
    metrics = results.metrics
else:
    metrics = None

if metrics is not None:
    # Cria um DataFrame com as métricas para facilitar a visualização
    df_metrics = pd.DataFrame([metrics])
    print("\nMétricas do Treinamento:")
    print(df_metrics)
else:
    print("\nNenhuma métrica estruturada encontrada no objeto results.")

# Se os dados das curvas estiverem disponíveis, plote os gráficos comparativos
if hasattr(results, 'curves') and hasattr(results, 'curves_results'):
    curves = results.curves  # Lista com os nomes das curvas
    curves_data = results.curves_results  # Lista com os dados das curvas; cada item é [x_values, y_values]
    n_curves = len(curves)

    # Cria uma figura com um subplot para cada curva
    fig, axs = plt.subplots(1, n_curves, figsize=(5 * n_curves, 4))
    # Se tiver só uma curva, convertemos o objeto de subplot para uma lista para iterar
    if n_curves == 1:
        axs = [axs]

    for i in range(n_curves):
        # Os dados podem vir como arrays numpy
        x_data = curves_data[i][0]
        y_data = curves_data[i][1]
        axs[i].plot(x_data, y_data, marker="o", linestyle="-")
        axs[i].set_title(curves[i])
        axs[i].set_xlabel("Eixo X")
        axs[i].set_ylabel("Eixo Y")
        axs[i].grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("Dados para plotar curvas não encontrados no objeto results.")

# Realiza uma avaliação mais detalhada no conjunto de validação
print("\nRealizando avaliação no conjunto de validação...")
# Dependendo da versão do ultralytics, o método pode ser 'val' ou 'validate'
val_results = model.val()
print("\nMétricas de Validação:")
print(val_results)

# Se desejar extrair as métricas de validação (por exemplo, do atributo results_dict) em uma tabela:
if hasattr(val_results, 'results_dict'):
    df_val_metrics = pd.DataFrame([val_results.results_dict])
    print("\nTabela com Métricas de Validação:")
    print(df_val_metrics)
else:
    print("As métricas detalhadas de validação não estão disponíveis de forma estruturada.")


Codigo de Visão computacional em ação

In [ ]:
# Instala a biblioteca ultralytics, se ainda não estiver instalada
!pip install ultralytics

# Agora importe os módulos necessários
from google.colab import files
import os
from ultralytics import YOLO
from IPython.display import HTML, display
from base64 import b64encode

# Carrega o modelo treinado (ajuste o caminho se necessário)
model = YOLO('runs/detect/train/weights/best.pt')

# Solicita o upload do vídeo do seu computador
print("Por favor, faça upload do arquivo de vídeo:")
uploaded = files.upload()

# Obtém o nome do arquivo de vídeo enviado
video_filename = list(uploaded.keys())[0]
print("Vídeo enviado:", video_filename)

# Aplica o modelo YOLO no vídeo para detectar os cachos de uva, salvando automaticamente os resultados
results_video = model.predict(source=video_filename, show=True, save=True)

print("Processamento finalizado! Os vídeos anotados foram salvos na pasta 'runs/detect/predict/'.")

# Define o diretório de saída onde os resultados foram salvos
output_dir = 'runs/detect/predict'
output_video = None

# Procura por um arquivo com extensão .mp4 na pasta de saída
for file in os.listdir(output_dir):
    if file.lower().endswith(".mp4"):
        output_video = os.path.join(output_dir, file)
        break

if output_video is None:
    print("Arquivo de vídeo anotado não encontrado em", output_dir)
else:
    # Lê e exibe o vídeo anotado no notebook
    with open(output_video, "rb") as video_file:
        mp4 = video_file.read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f"""
    <video width="600" controls>
        <source src="{data_url}" type="video/mp4">
    </video>
    """))
